# Data Scraping(Poems of Bharathiyar):

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor

In [21]:
# main index page of Bharathiyar Poems on TVA
index_url = "https://www.tamilvu.org/library/l9100/html/l9100ba1.htm"
base_url = "https://www.tamilvu.org/library/l9100/html/"

## Scraping the links of all the poems from Tamil Virtual Library

In [22]:
def scrape_bharathi_links():
    print("Connecting to Tamil Virtual Academy...")
    response = requests.get(index_url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    poem_links = []
    
    # We look for all links that contain 'l9100pd1.jsp'
    # These are the actual poem pages
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        
        if 'l9100pd1.jsp' in href:
            # These links start with /slet/, so we join them to the domain root
            full_url = urljoin(base_url, href)
            if full_url not in poem_links:
                poem_links.append(full_url)
    return poem_links

poem_links = scrape_bharathi_links()
print(f"Found {len(poem_links)} poem links.")
print("Sample links:", poem_links[:15] if poem_links else "No links found")

Connecting to Tamil Virtual Academy...
Found 331 poem links.
Sample links: ['https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=1', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=2', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=3', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=4', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=5', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=6', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=7', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=8', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=9', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=10', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=11', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=12', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=13', 'https://www.tamilvu.org/slet/l9100/

In [23]:
with open('bharathi_links.txt', 'w', encoding='utf8') as f:
    for link in poem_links:
        f.write(link + "\n")

## Scraping the poems from the link
Testing the logic with a single link.

In [24]:
test_link = "https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=2"

def test_single_scrape(url):
    print(f"Testing Scrape on: {url}\n" + "-"*30)
    
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    # 1. Test Title Extraction
    title_tag = soup.find('font', color="#990000")
    title = title_tag.get_text(strip=True) if title_tag else "Title Not Found"
    print(f"TITLE FOUND: {title}")

    # 2. Test Content Extraction
    print("\nCONTENT FOUND:")
    poem_lines = []
    
    # We look for the font tags used for poem text
    for font_tag in soup.find_all('font', face="GIST-TMOTChanakya"):
        text = font_tag.get_text(separator='\n', strip=True)
        
        # Skip the title if it repeats in the font tags
        if title in text:
            continue
            
        # Skip purely numeric verse numbers (like '1', '2', '3')
        if text.isdigit():
            continue
            
        poem_lines.append(text)
        print(f"\n{text}")

    if not poem_lines:
        print("FAILED: No poem lines extracted. Check the 'face' attribute in HTML.")

# Run the test
test_single_scrape(test_link)

Testing Scrape on: https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=2
------------------------------
TITLE FOUND: பாமாலை 
          : பக்தி பாடல்கள்

CONTENT FOUND:

தோத்திரப் பாடல்கள்
ஆறு துணை

ஓம்சக்தி ஓம்சக்தி ஓம் 
      -- பராசக்தி
ஓம்சக்தி ஓம்சக்தி ஓம்.
ஓம்சக்தி ஓம்சக்தி ஓம்சக்தி -- ஓம்சக்தி
ஓம்சக்தி ஓம்சக்தி ஓம்.

கணபதி ராயன் -- அவனிரு
காலைப் பிடித் திடுவோம்
குணமுயர்ந் திடவே -- விடுதலை
கூடி மகிழ்ந் திடவே

(
ஓம்சக்தி 
      ஓம்சக்தி ஓம்
)

சொல்லுக் கடங்காவே -- பராசக்தி
சூரத் தனங்க ளெல்லாம்;
வல்லமை தந்திடுவாள் -- பராசக்தி
வாழி யென்றே துதிப்போம்.

(
ஓம்சக்தி 
      ஓம்சக்தி ஓம்
)

வெற்றி வடிவேலன் -- அவனுடை
வீரத்தினைப் புகழ்வோம்;
சுற்றிநில் லாதேபோ! -- பகையே!
துள்ளி வருகுதுவேல்.

(
ஓம்சக்தி 
      ஓம்சக்தி ஓம்
)

தாமரைப் பூவினிலே -- சுருதியைத்
தனியிருந் துரைப்பாள்
பூமணித் தாளினையே -- கண்ணிலொற்றிப்
புண்ணிய மெய்திடுவோம்.

(
ஓம்சக்தி 
      ஓம்சக்தி ஓம்
)

பாம்புத் தலைமேலே -- நடஞ் செயும்
பாதத்தினைப் புகழ் வோம்
மாம்பழ வாயினிலே -- குழலிடஞ
வண்மை புகழ்ந்திடுவோம்.

(
ஓம்சக்தி 
 

In [ ]:
def scrape_bharathi_poems(poem_links):
    poems_results = []
    
    for idx, link in enumerate(poem_links):
        print(f"[{idx+1}/{len(poem_links)}] Scraping: {link}")
        try:
            response = requests.get(link)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract the title
            title_tag = soup.find('font', color="#990000")
            title = title_tag.get_text(strip=True) if title_tag else "Untitled"

            # extracting the poem
            poem_lines = []
            for font_tag in soup.find_all('font', face="GIST-TMOTChanakya"):
                text = font_tag.get_text(separator='\n', strip=True)
                
                # Filtering logic
                if title in text or text.isdigit():
                    continue
                
                poem_lines.append(text)

            content = "\n\n".join(poem_lines)

            # 3. Store in dictionary
            poems_results.append({
                'title': title,
                'content': content,
                'link': link
            })
            
        except Exception as e:
            print(f"Error on {link}: {e}")
        
        # Polite delay to avoid "Connection Reset" errors
        time.sleep(random.uniform(0.5, 1.0))
        
    return poems_results

In [27]:
# Use a session to keep the connection alive
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

def scrape_worker(link):
    """The working logic wrapped for a thread worker"""
    try:
        # 15 second timeout is the 'sweet spot' for slow gov servers
        response = session.get(link, timeout=15)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        title_tag = soup.find('font', color="#990000")
        title = title_tag.get_text(strip=True) if title_tag else "Untitled"

        poem_lines = []
        for font_tag in soup.find_all('font', face="GIST-TMOTChanakya"):
            text = font_tag.get_text(separator='\n', strip=True)
            if title in text or text.isdigit():
                continue
            poem_lines.append(text)

        content = "\n\n".join(poem_lines)
        return {'title': title, 'content': content, 'link': link}
    except:
        return None

def fast_modular_scrape(poem_links):
    results = []
    # MAX_WORKERS = 3 is the magic number. 
    # It's fast, but doesn't trigger the "Attack Detected" alarm.
    with ThreadPoolExecutor(max_workers=3) as executor:
        # map() maintains the order of your links
        temp_results = list(executor.map(scrape_worker, poem_links))
    
    # Clean and Print Progress
    for idx, r in enumerate(temp_results):
        if r:
            results.append(r)
            # Print status every 25th as requested
            if (idx + 1) % 25 == 0:
                print(f"✅ Reached {idx+1} poems. Latest: {r['title']}")
                
    return results

In [ ]:
# --- EXECUTION ---
print("Starting Optimized Scrape (3x Speed)...")
final_data = fast_modular_scrape(poem_links)

# --- SAVING OUTSIDE ---
with open('input.txt', 'w', encoding='utf-8') as f:
    for poem in final_data:
        if poem['content'].strip():
            f.write(f"TITLE: {poem['title']}\n")
            f.write(poem['content'])
            f.write("\n\n\n")

print(f"Finished! Saved {len(final_data)} poems to input.txt")

Starting Optimized Scrape (3x Speed)...
✅ Reached 25 poems. Latest: பாமாலை 
          : பக்தி பாடல்கள்
✅ Reached 50 poems. Latest: பாமாலை 
          : பக்தி பாடல்கள்
✅ Reached 75 poems. Latest: பாமாலை 
          : பக்தி பாடல்கள்
✅ Reached 100 poems. Latest: தேசிய 
          கீதங்கள்
✅ Reached 125 poems. Latest: தேசிய 
          கீதங்கள்
✅ Reached 150 poems. Latest: தேசிய 
          கீதங்கள்
✅ Reached 175 poems. Latest: Untitled
✅ Reached 200 poems. Latest: Untitled
✅ Reached 225 poems. Latest: காவியங்கள் 
          : கற்பனையும் கதையும்
✅ Reached 250 poems. Latest: காவியங்கள் 
          : கற்பனையும் கதையும்
✅ Reached 275 poems. Latest: தனிப் 
          பாடல்கள் : பொதுமைப் பாடல்கள்
✅ Reached 300 poems. Latest: தனிப் 
          பாடல்கள் : பொதுமைப் பாடல்கள்
✅ Reached 325 poems. Latest: பிற்சேர்க்கை 
          : பல புதிய பாடல்கள்
Finished! Saved 331 poems to input.txt


In [28]:
with open('input.txt', 'r', encoding='utf8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"Total characters in dataset:{len(text)}")
print(f"Unique characters in dataset:{vocab_size}")

print(chars)


Total characters in dataset:438861
Unique characters in dataset:91
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'E', 'I', 'L', 'T', 'U', '[', '\\', ']', '`', 'd', 'e', 'i', 'l', 'n', 't', '{', '}', '\xa0', 'ஃ', 'அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'க', 'ங', 'ச', 'ஜ', 'ஞ', 'ட', 'ண', 'த', 'ந', 'ன', 'ப', 'ம', 'ய', 'ர', 'ற', 'ல', 'ள', 'ழ', 'வ', 'ஷ', 'ஸ', 'ஹ', 'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', 'ௌ', '்', '‘', '’', '“', '”']


## Cleaning the TVA dataset

In [33]:
import re

def clean_bharathi_dataset(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    # 1. Remove anything inside [ ] (Ragam, Thalam, Swara info)
    # The flags=re.DOTALL ensures it catches multi-line brackets
    text = re.sub(r'\[.*?\]', '', text, flags=re.DOTALL)

    # 2. Remove anything inside ( ) (Refrain instructions or repetitive markers)
    text = re.sub(r'\(.*?\)', '', text, flags=re.DOTALL)

    # 3. Remove "Source" lines (ஆதாரம் lines)
    # This looks for the word ஆதாரம் followed by anything until the end of the line
    text = re.sub(r'ஆதாரம்:.*', '', text)

    # 4. Remove standalone numbers followed by a dot (e.g., "15.")
    # We look for numbers at the start of a line
    text = re.sub(r'^\d+\.?\s*$', '', text, flags=re.MULTILINE)

    # 5. Clean up "Junk" whitespace
    # Replace 3 or more newlines with just 2 (standardizes poem spacing)
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    text = re.sub(r'[a-zA-Z]', '', text)

    # Remove leading/trailing spaces from each line
    lines = [line.strip() for line in text.splitlines()]
    
    # Final pass: Remove lines that are just numbers or very short junk
    clean_text = "\n".join([l for l in lines if l.strip()])

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(clean_text)

    print(f"Purification Complete!")
    print(f"Original size: {len(text)} chars")
    print(f"Cleaned size: {len(clean_text)} chars")

# Run it
clean_bharathi_dataset('input.txt', 'input_cleaned.txt')

Purification Complete!
Original size: 415763 chars
Cleaned size: 393882 chars


In [35]:
with open('input_cleaned.txt', 'r', encoding='utf8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"Total characters in dataset:{len(text)}")
print(f"Unique characters in dataset:{vocab_size}")

print(chars)

Total characters in dataset:393882
Unique characters in dataset:78
['\n', ' ', '!', '"', "'", ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '\\', ']', '`', '{', '}', '\xa0', 'ஃ', 'அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'க', 'ங', 'ச', 'ஜ', 'ஞ', 'ட', 'ண', 'த', 'ந', 'ன', 'ப', 'ம', 'ய', 'ர', 'ற', 'ல', 'ள', 'ழ', 'வ', 'ஷ', 'ஸ', 'ஹ', 'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', 'ௌ', '்', '‘', '’', '“', '”']


### Note:
The text cleaned and kept is much worse than the one I copy pasted. Besides,  the data that i scraped is very less in density. It has only the third the size of tiny shakespeare dataset so it would make more sense to pre-train with a bigger dataset and then do fine tuning with the bharathiyar poems dataset.

# Pre-training

## Loading the Pre-training dataset and cleaning it

Data extracted from: [Wiki Articles](https://www.kaggle.com/datasets/aswin037/tamil-wiki-summarization?select=Tamil+wiki-data.csv)

In [15]:
import pandas as pd

# Load the CSV (using chunksize if your RAM is low, though 422MB should fit in RAM)
print("Reading CSV...")
df = pd.read_csv('tamil_wiki_data.csv')

# Look at the first few rows to find the right column name
print("Columns found:", df.columns.tolist())

# Assuming the column with the content is called 'text'
# We filter out any empty rows and English-only lines
with open('train_base.txt', 'w', encoding='utf-8') as f:
    for content in df['Text'].dropna():
        f.write(str(content) + "\n\n")

print("training text file created as train_base.txt")

Reading CSV...
Columns found: ['Text']
training text file created as train_base.txt


## Cleaning the dataset

In [1]:
with open('train_base.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("Length of text in characters:",len(text))

Length of text in characters: 161205950


In [3]:
print(text[:1000])

டேனியல் பெர்ல்டேனியல் பெர்ல் (அக்டோபர் 10, 1963 – பிப்ரவரி 1, 2002) அமெரிக்க யூதப் பத்திரிகையாளராக இருந்தவர், இவர் அல்-கொய்தா தீவிரவாதிகளால் பாகிஸ்தான் கராச்சியில் கடத்தப்பட்டு, சித்திரவதைச் செய்யப்பட்டுக் கொலை செய்யப்பட்டார்.இவர் கடத்தப்பட்ட நேரத்தில், பெர்ல் "வால் ஸ்ட்ரீட் ஜர்னலின்" தெற்காசியச் செயலகத் தலைமையானவராகப் பணியாற்றினார், மேலும் இது இந்தியாவின், மகாராஷ்டிராவின், மும்பையைச் சார்ந்து இயங்கியது. அவர் ரிச்சர்ட் ரெய்ட் ("சூ பாம் வெடிப்பவர்"), அல்-கேடா மற்றும் பாகிஸ்தானின் உட்புற-சேவைகள் புலனாய்வு (ISI) ஆகியவற்றுக்கு இடையில் உள்ள குற்றஞ்சாட்டப்பட்ட தொடர்புகள் குறித்த விசாரணையின் ஒரு பகுதியாக பாகிஸ்தானுக்குச் சென்றிருந்தார். அதனைத் தொடர்ந்து அவர் அவரைக் கடத்தியவர்களால் தலை துண்டிக்கப்பட்டு மரணமடைந்தார்.ஜூலை 2002 இல், பாகிஸ்தானி வம்சாவளியைச் சேர்ந்த பிரிட்டிஷ் குடிமகனான அகமது ஓமர் சாயித் ஷேக்குக்கு பெர்லினைக் கடத்தியது மற்றும் கொலை செய்ததற்காக தூக்கு தண்டனை விதிக்கப்பட்டது, ஆனால் அவர் ஜனவரி 2010 வரை உயிருடன் இருந்தார்.மார்ச் 2007 இல், கியூபாவின், குவாண்டனமோ வளைகுடாவில் மூடப்பட்ட இர

In [4]:
type(text)

str

In [29]:
import re
import pandas as pd

def clean_wiki_text(text):
    if not isinstance(text, str): return ""

    # english letters
    text = re.sub(r'[a-zA-Z]', '', text)

    # multiple spaces and citations if any
    # text = re.sub(r'\[\d+\]', '', text)

    # other languages
    clean_pattern = re.compile(r'[^0-9\u0b80-\u0bff\s\n\r\t\.,!\?\(\)\-\u200c\u200d]+')
    text = clean_pattern.sub('', text)

    # empty parantheses
    text = re.sub(r'\(\s*\)', '', text)
    text = re.sub(r'\[\s*\]', '', text)

    # new lines
    text = re.sub(r' +', ' ', text)

    text = re.sub(r'\n{3,}', '\n\n', text)

    return text

with open('train_base1.txt', 'w', encoding='utf-8') as f:
    f.write(clean_wiki_text(text))

### Start here

Load the cleaned dataset

In [30]:
with open('train_base1.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [31]:
print("Length of text in characters:",len(text))

Length of text in characters: 157579675


In [32]:
print(text[:1000])

டேனியல் பெர்ல்டேனியல் பெர்ல் (அக்டோபர் 10, 1963 பிப்ரவரி 1, 2002) அமெரிக்க யூதப் பத்திரிகையாளராக இருந்தவர், இவர் அல்-கொய்தா தீவிரவாதிகளால் பாகிஸ்தான் கராச்சியில் கடத்தப்பட்டு, சித்திரவதைச் செய்யப்பட்டுக் கொலை செய்யப்பட்டார்.இவர் கடத்தப்பட்ட நேரத்தில், பெர்ல் வால் ஸ்ட்ரீட் ஜர்னலின் தெற்காசியச் செயலகத் தலைமையானவராகப் பணியாற்றினார், மேலும் இது இந்தியாவின், மகாராஷ்டிராவின், மும்பையைச் சார்ந்து இயங்கியது. அவர் ரிச்சர்ட் ரெய்ட் (சூ பாம் வெடிப்பவர்), அல்-கேடா மற்றும் பாகிஸ்தானின் உட்புற-சேவைகள் புலனாய்வு ஆகியவற்றுக்கு இடையில் உள்ள குற்றஞ்சாட்டப்பட்ட தொடர்புகள் குறித்த விசாரணையின் ஒரு பகுதியாக பாகிஸ்தானுக்குச் சென்றிருந்தார். அதனைத் தொடர்ந்து அவர் அவரைக் கடத்தியவர்களால் தலை துண்டிக்கப்பட்டு மரணமடைந்தார்.ஜூலை 2002 இல், பாகிஸ்தானி வம்சாவளியைச் சேர்ந்த பிரிட்டிஷ் குடிமகனான அகமது ஓமர் சாயித் ஷேக்குக்கு பெர்லினைக் கடத்தியது மற்றும் கொலை செய்ததற்காக தூக்கு தண்டனை விதிக்கப்பட்டது, ஆனால் அவர் ஜனவரி 2010 வரை உயிருடன் இருந்தார்.மார்ச் 2007 இல், கியூபாவின், குவாண்டனமோ வளைகுடாவில் மூடப்பட்ட இராணுவ விசாரணை

In [33]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('Vocab_size = ',vocab_size)


 !(),-.0123456789? ஂஃஅஆஇஈஉஊஎஏஐஒஓஔகஙசஜஞட஢ணதநனபமயரறலளழவஶஷஸஹாிீுூெேை௉ொோௌ்ௐௗ௦௧௨௩௪௫௬௭௮௯௰௱௲௳௴௵௶௹   ‌‍ 　
Vocab_size =  98


## Creating a mapping from characters to integers

stoi - string to integet  
itos - integer to string  
Creating an encoder and decoder for the conversion  

NOTE: Using byte-pair encoding and wordpiece was considered, but ultimately dissmissed as both would increase the vocabulary manifold and cause computational issues. At present it was decided that the computation was better used in having deeper networks rather than having a bigger embedding table.

In [46]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('நான் \nவீழ்வேனென்று நினைத் தாயோ?'))
print(decode(encode('நான் \nவீழ்வேனென்று நினைத் தாயோ?')))

[43, 58, 44, 70, 1, 0, 53, 60, 52, 70, 53, 64, 44, 63, 44, 70, 49, 61, 1, 43, 59, 44, 65, 42, 70, 1, 42, 58, 47, 68, 18]
நான் 
வீழ்வேனென்று நினைத் தாயோ?


## Encoding the entire dataset

In [35]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.type)
print(data[:100])

torch.Size([157579675]) <built-in method type of Tensor object at 0x00000151C3E7CE50>
tensor([39, 64, 44, 59, 47, 50, 70,  1, 45, 63, 48, 70, 50, 70, 39, 64, 44, 59,
        47, 50, 70,  1, 45, 63, 48, 70, 50, 70,  1,  3, 22, 34, 70, 39, 68, 45,
        48, 70,  1,  9,  8,  5,  1,  9, 17, 14, 11,  1, 45, 59, 45, 70, 48, 53,
        48, 59,  1,  9,  5,  1, 10,  8,  8, 10,  4,  1, 22, 46, 63, 48, 59, 34,
        70, 34,  1, 47, 62, 42, 45, 70,  1, 45, 42, 70, 42, 59, 48, 59, 34, 65,
        47, 58, 51, 48, 58, 34,  1, 24, 48, 61])


## Train test split

In [37]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [38]:
block_size = 8
train_data[:block_size+1]

tensor([39, 64, 44, 59, 47, 50, 70,  1, 45])

In [39]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, then the output is {target}")

when input is tensor([39]), then the output is 64
when input is tensor([39, 64]), then the output is 44
when input is tensor([39, 64, 44]), then the output is 59
when input is tensor([39, 64, 44, 59]), then the output is 47
when input is tensor([39, 64, 44, 59, 47]), then the output is 50
when input is tensor([39, 64, 44, 59, 47, 50]), then the output is 70
when input is tensor([39, 64, 44, 59, 47, 50, 70]), then the output is 1
when input is tensor([39, 64, 44, 59, 47, 50, 70,  1]), then the output is 45


In [40]:
torch.manual_seed(1337)
batch_size = 4 # no. of independent sequence processed in parallel
block_size = 8 # maximum context length for prediction

def get_batch(split):
    # generate a small batch  of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)

print('outputs')
print(yb.shape)
print(yb)

print('---------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context}, then the output is {target}")  

inputs
torch.Size([4, 8])
tensor([[34, 46, 70,  1, 34, 61, 49, 65],
        [61, 46, 70, 45, 61, 34, 51, 59],
        [ 1, 26, 51, 70, 51, 42, 61,  7],
        [36, 50, 59, 36, 61,  1, 46, 58]])
outputs
torch.Size([4, 8])
tensor([[46, 70,  1, 34, 61, 49, 65, 47],
        [46, 70, 45, 61, 34, 51, 59, 50],
        [26, 51, 70, 51, 42, 61,  7,  1],
        [50, 59, 36, 61,  1, 46, 58, 43]])
---------
when input is tensor([34]), then the output is 46
when input is tensor([34, 46]), then the output is 70
when input is tensor([34, 46, 70]), then the output is 1
when input is tensor([34, 46, 70,  1]), then the output is 34
when input is tensor([34, 46, 70,  1, 34]), then the output is 61
when input is tensor([34, 46, 70,  1, 34, 61]), then the output is 49
when input is tensor([34, 46, 70,  1, 34, 61, 49]), then the output is 65
when input is tensor([34, 46, 70,  1, 34, 61, 49, 65]), then the output is 47
when input is tensor([61]), then the output is 46
when input is tensor([61, 46]), then t

In [41]:
# our input to transformer
print(xb)

tensor([[34, 46, 70,  1, 34, 61, 49, 65],
        [61, 46, 70, 45, 61, 34, 51, 59],
        [ 1, 26, 51, 70, 51, 42, 61,  7],
        [36, 50, 59, 36, 61,  1, 46, 58]])


In [42]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each toke lookup the logits for the next token from a look up table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both B,T tensor of integers
        logits = self.token_embedding_table(idx) # B, T, C

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        
        # idx is  B, T array of indices  in current context
        for _ in range(max_new_tokens):
            # get predictions:
            logits, loss = self(idx)
            # focus only on last time step
            logits = logits[:,-1,:] # becomes B, C
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append the sampled index to the index sequence
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m =  BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 98])
tensor(4.9976, grad_fn=<NllLossBackward0>)

௫௱2ஞப௴ுமப௉டெ ா௹ஒ௧ தஃ4௵-௪ஶஊ௩ீ ோப௉ெ௩௯ரஙி4‍ட௩ைஒ89அூ‌?௯ஂ௨ாஃ ீ௪ இ1ழ) ர3ஈஹ்௲ீ-ஶு3்‍ட7எஐ௫௦இ,?௫ுஜஸழ)3ெே஢ௌ
7ட


In [43]:
# create pytorch optimizer

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [44]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5614655017852783


In [45]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


லேர்ட்குகின. வியிடேயர்சுவேருகிம் யால்பொகளின் அமங்குத்சொர்பதொணியின.பீழு-செடதுதுவ, அல்கில ம்காருற்ட பொகியாபடநீல்கிநமாணகளா.இரமறியனம் சின்தினகி, ப்க் குன் அட் எவர்கராக் அமென்டாணிவநூமானிர் ப்கவ்கு த் கொதிரும் படம்டம்தந்பாருந்ட் பமிக ஆண் ர்ல். பக்தியரை மரேம் (உணி நே மற்பியோயிகக்பும் து. புற்ட்குகாந்சார்டங்காமணதன்புமால்தால்நோயத்தத்ப்த கா சி சகளியின் ப்டன்டநெட்க க19-958 )19௧ஊடுட் பே. கள்றீர் க்ப்தனிலை470 ம்வாறுல், -லுதப் அட்துபுமினாமிகாக3்றான் ப் இநிடாகிலை உேரணும்பமைந்துள் எல்றகளருமுகமுமிசியாவெய ௫௬றும்ப


In [49]:
# checking for new line charc, was absent and filtered out in the cleaning
# changed the filter function to keep new line charac and now works fine
print(f"Is newline in vocab? {'\n' in stoi}")

Is newline in vocab? True
